<a href="https://colab.research.google.com/github/afrahnaaz/AI/blob/main/Assignment-4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

mount google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import libraries

In [ ]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt

load data

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/DATA_AI/Ground_water_data.csv')
data.head()

outliners treatment

In [ ]:
outliers = []
def detect_outliers_zscore(data):
  thres = 3
  mean = np.mean(data)
  std = np.std(data)
  for i in data:
    z_score = (i - mean)/std
    if(np.abs(z_score) > thres):
      outliers.append(i)
  return outliers

In [ ]:
sample_outliers_X = detect_outliers_zscore(data['X'])
sample_outliers_Y = detect_outliers_zscore(data['Y'])
print("Outliers in X column from Z-scores method: ", sample_outliers_X)
print("Outliers in Y column from Z-scores method: ", sample_outliers_Y)

Missing values treatment

In [ ]:
print(data.isnull().sum())

In [ ]:
m1=data['X'].mean()
m2=data['Y'].mean()
data['X'].fillna(value=m1, inplace=True)
data['Y'].fillna(value=m2, inplace=True)

Min and Max values

In [ ]:
x_min = data['X'].min()
x_max = data['X'].max()
y_min = data['Y'].min() 
y_max = data['Y'].max()
print(x_min, x_max, y_min, y_max) 

Normalisation

In [ ]:
from sklearn.preprocessing import MinMaxScaler
X1 = np.array(data['X']).reshape(-1,1)
scaler = MinMaxScaler()
scaler.fit(X1)
X_scaled = scaler.transform(X1)
data['X'] = X_scaled.reshape(1,-1)[0]

In [ ]:
Y1 = np.array(data['Y']).reshape(-1,1)
scaler = MinMaxScaler()
scaler.fit(Y1)
Y_scaled = scaler.transform(Y1)
data['Y'] = Y_scaled.reshape(1,-1)[0]

Data Split

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data.X, data.Y, test_size = 0.10, random_state = 10)

Initialisation

In [ ]:
m = random.uniform(-10, 10)
c = random.uniform(-10, 10)
eta = 0.9
epochs = 2000

Gradient Calculation

In [ ]:
def gradient(m, c, x, y):
  m_deriv = -1*(y - m*x -c)*x;
  c_deriv = -1*(y - m*x - c)
  return m_deriv, c_deriv

Training the Model

In [ ]:
for iter in range(epochs):
  for sample in range(x_train.shape[0]):
    der_m, der_c = gradient(m, c, x_train.iloc[sample], y_train.iloc[sample])
    m = m - eta*der_m
    c = c - eta*der_c

print('Model Parameters are ', m, c)

Model Prediction on training data

In [ ]:
y_pred_train = []
for i in x_train:
  y_p_tr = (m*i) + c
  y_pred_train.append(y_p_tr)
y_pred_train = np.array(y_pred_train)
print(y_pred_train)

Model Prediction on Testing Data

In [ ]:
y_pred_test = []
for i in x_test:
  y_pr_te = (m*i) + c
  y_pred_test.append(y_pr_te)
y_pred_test = np.array(y_pred_test)
print(y_pred_test)

Error Metrics Calculation

In [ ]:
import math
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import mean_absolute_error

# Training Accuracy
rmse = math.sqrt(mean_squared_error(y_train, y_pred_train)) 
print('Root mean square error', rmse)
mse = (mean_squared_error(y_train, y_pred_train)) 
print('Mean square error', mse)
mae=mean_absolute_error(y_train, y_pred_train)
print('Mean absolute error', mae)

In [ ]:
#Testing Accuracies
mse = math.sqrt(mean_squared_error(y_test, y_pred_test)) 
print('Root mean square error', mse) 
mse = (mean_squared_error(y_test, y_pred_test)) 
print('Mean square error', mse) 
mae=mean_absolute_error(y_test, y_pred_test)
print('Mean absolute error', mae)

Deployment

In [ ]:
X=float(input('Enter PH Value '))
print('PH Value =',X)

x_nor = (X - x_min)/(x_max - x_min)
y_nor = m*x_nor + c

Y = y_nor*(y_max - y_min) + y_min 
print('Bicarbobates of well water = ', np.round(Y))